In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

from Constants.Paths import *
from Constants.Labels import *
from Unpacking.PrepareAudioFiles import prepare_audio_files
from Preprocessing.GenerateSpectrograms import generate_all_spectrograms
from SpectrogramLoading import *
from Models.TrainingHistory import TrainingHistory
from Models.CnnModel import CnnModel
from Models.WandbDetails import WandbDetails
from Models.InputPadding import pad_to_length
from Models.HistoryPlots import plot_loss_history, plot_accuracy_history

In [2]:
# Before running this notebook, do the following:
# - Unzip 'tensorflow-speech-recognition-challenge.zip' to 'Dataset' folder
# - Remove everything except 'train.7z' and 'test.7z'
# - Unzip 'train.7z' and 'test.7z' also to 'Dataset' folder (not to new 'train'/'test' folders)
# - Make sure that there is only one level of folders named 'train'/'test' (not 'Dataset/train/train/...')

In [ ]:
prepare_audio_files()
generate_all_spectrograms(backend="soundfile")

In [4]:
train_paths, val_paths = get_divided_paths_with_labels()

random.seed(42)
random.shuffle(train_paths)
random.shuffle(val_paths)

#train_paths = train_paths[:1000]
#val_paths = val_paths[:1000]

train = [load_spectrogram_for_path(path_with_label) for path_with_label in train_paths]
validation = [load_spectrogram_for_path(path_with_label) for path_with_label in val_paths]

In [5]:
X_train, y_train, label_indexes = spectrograms_to_x_y(train)
X_validation, y_validation, _ = spectrograms_to_x_y(validation, label_indexes)

max_length = max(x.shape[1] for x in [*X_validation, *X_train])
X_validation = pad_to_length(X_validation, max_length)
X_train = pad_to_length(X_train, max_length)

In [10]:
from Models.TransformerModel import TransformerModel

transformer_model = TransformerModel(
    classes=labels,
    learning_rate=1e-4,
    lr_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    eps=1e-8,
    # wandb_details=WandbDetails(
    #     project="dl-speech-recognition",
    #     experiment_name="transformer-test",
    #     config_name="first-transformer",
    #     artifact_name="transformer-model"
    # )
)

In [7]:
cnn_model = CnnModel(
    classes=labels,
    learning_rate=1e-4,
    lr_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    eps=1e-8,
    classifier_dropout_1=0.1,
    classifier_dropout_2=0.1,
    classifier_dropout_3=0.1,
    wandb_details=WandbDetails(
        project="dl-speech-recognition",
        experiment_name="integration-test-3",
        config_name="first-cnn",
        artifact_name="test-model"
    )
)

In [ ]:
transformer_model.train((X_train, y_train), (X_validation, y_validation), epochs=10, batch_size=32)

In [ ]:
_, ax = plt.subplots(2, 1, figsize=(10, 11))
plot_loss_history(transformer_model.get_history(), ax[0])
plot_accuracy_history(transformer_model.get_history(), ax[1])
plt.show()

In [ ]:
import optimization
from skopt.space import Real


def evaluate_cnn_params(**params):
    cnn_model = CnnModel(
        classes=labels,
        learning_rate=params["learning_rate"],
        lr_decay=params["lr_decay"],
        beta_1=1-params["inv_beta_1"],
        beta_2=1-params["inv_beta_2"],
        eps=params["eps"],
        classifier_dropout_1=params["classifier_dropout_1"],
        classifier_dropout_2=params["classifier_dropout_2"],
        classifier_dropout_3=params["classifier_dropout_3"],
        print_every=None # Disable printing epoch info
    )
    
    # TODO: We robably need more epochs in real search
    cnn_model.train((X_train, y_train), (X_validation, y_validation), epochs=10, batch_size=32)
    
    # Use last validation accuracy as metric - could also evaluate on test set directly
    return cnn_model.get_history().val_accuracy[-1] 

spaces = {
    "learning_rate": Real(1e-6, 1e-2, "log-uniform"),
    "lr_decay": Real(1e-6, 1e-2, "log-uniform"),
    "inv_beta_1": Real(1e-4, 0.5, "log-uniform"),
    "inv_beta_2": Real(1e-6, 0.1, "log-uniform"),
    "eps": Real(1e-8, 1e-3, "log-uniform"),
    "classifier_dropout_1": Real(0.0, 0.3, "uniform"),
    "classifier_dropout_2": Real(0.0, 0.3, "uniform"),
    "classifier_dropout_3": Real(0.0, 0.3, "uniform")
}

best_params, best_accuracy = optimization.bayes_search(evaluate_cnn_params, spaces, iterations=10)